<a href="https://colab.research.google.com/github/tejatammali/Twitter-Sentiment-Analysis/blob/main/twitter_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#libraries
import re 
import tweepy 
from tweepy import OAuthHandler 
from textblob import TextBlob 
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import matplotlib.pyplot as plt

In [ ]:
#class
class TwitterClient(object): 
    ''' 
    Generic Twitter Class for sentiment analysis. 
    '''
    def __init__(self): 
        ''' 
        Class constructor or initialization method. 
        '''
        # keys and tokens from the Twitter Dev Console 
        consumer_key = 'xxxxxxxxxxxxxxxxxxxxxxxxx'
        consumer_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
        access_token = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
        access_token_secret = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'
  
        # attempt authentication 
        try: 
            # create OAuthHandler object 
            self.auth = OAuthHandler(consumer_key, consumer_secret) 
            # set access token and secret 
            self.auth.set_access_token(access_token, access_token_secret) 
            # create tweepy API object to fetch tweets 
            self.api = tweepy.API(self.auth) 
        except: 
            print("Error: Authentication Failed") 
  
    def clean_tweet(self, tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split()) 
  
    def get_tweet_sentiment(self, tweet): 
        ''' 
        Utility function to classify sentiment of passed tweet 
        using textblob's sentiment method 
        '''
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(self.clean_tweet(tweet)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'
  
    def get_tweets(self, query, count = 10): 
        ''' 
        Main function to fetch tweets and parse them. 
        '''
        # empty list to store parsed tweets 
        tweets = [] 
  
        try: 
            # call twitter api to fetch tweets 
            fetched_tweets = self.api.search(q = query, count = count) 
  
            # parsing tweets one by one 
            for tweet in fetched_tweets: 
                # empty dictionary to store required params of a tweet 
                parsed_tweet = {} 
  
                # saving text of tweet
                parsed_tweet['text'] = tweet.text 
                # saving sentiment of tweet 
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text) 
  
                # appending parsed tweet to tweets list 
                if tweet.retweet_count > 0: 
                    # if tweet has retweets, ensure that it is appended only once 
                    if parsed_tweet not in tweets: 
                        tweets.append(parsed_tweet) 
                else: 
                    tweets.append(parsed_tweet) 
  
            # return parsed tweets 
            return tweets 
  
        except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e)) 

#main
def main(): 

  while True:
    # creating object of TwitterClient Class 
    api = TwitterClient() 
    # calling function to get tweets 
    
    print("-------------------------------------------------------------------------------------------\n")

    userInput = input("Enter query you would like to search: ")
    if userInput == 'done' or userInput == 'DONE' or userInput == 'stop' or userInput == "STOP":
      break

    title = userInput
    tweets = api.get_tweets(query = userInput, count = 200) 
  
    # picking positive tweets from tweets 
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {0:.2f}%".format(100*len(ptweets)/len(tweets))) 
    
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets
    print("Negative tweets percentage: {0:.2f}%".format(100*len(ntweets)/len(tweets))) 
    
    # percentage of neutral tweets
    print("Neutral tweets percentage: {0:.2f}%".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:")   
    for tweet in ntweets[:10]: 
        print(tweet['text'])

    positiveOne = '{0:.2f}'.format((100*len(ptweets)/len(tweets)))
    negativeOne = '{0:.2f}'.format((100*len(ntweets)/len(tweets)))
    neutralOne = '{0:.2f}'.format((100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))

    print()

    #pie
    labels = ['Positive ['+str(positiveOne)+'%]', 'Negative ['+str(negativeOne)+'%]','Neutral ['+str(neutralOne)+'%]']
    sizes = [positiveOne, negativeOne, neutralOne]
    colors = ['blue','red','yellow']
    patches, texts = plt.pie(sizes, colors=colors, startangle=90)
    plt.legend(patches,labels,loc="best")
    plt.title("Type of Tweet and Percentage of Tone for the Query: " + title)
    plt.axis('equal')
    plt.tight_layout()
    plt.show()

    #bar
    data = [positiveOne, negativeOne, neutralOne]
    plt.barh(["positive","negative","neutral"], data, color='royalblue')
    plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='x', alpha=5)
    plt.ylabel('Type of Tweet', fontweight='bold', fontsize='13', horizontalalignment='center')
    plt.xlabel('Percentage of Tweet Type', fontweight='bold', fontsize='13', horizontalalignment='center')
    plt.title("Type of Tweet and Percentage of Tone for the Query: " + title, fontweight='bold', fontsize='13', horizontalalignment='center')
    plt.show()

if __name__ == "__main__": 
    # calling main function 
    main()